In [1]:
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"

spark = SparkSession.builder \
    .appName("jieba_test2") \
    .master("spark://node01:10077") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/18 10:13:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/18 10:13:16 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [30]:
import jieba
import jieba.analyse

stop_words_rdd = sc.textFile("hdfs:///user/spark_temp/stopwords.dat")
stop_words_set = set(stop_words_rdd.collect())

def cut_name_and_desc(item):
    id, name, desc = item
    name_cut = set(jieba.cut(name))
    name_cut_pure = set(jieba.cut(name)) - stop_words_set
    desc_cut = set(jieba.cut(desc))
    desc_cut_pure = set(jieba.cut(desc)) - stop_words_set
    return (id,name_cut,name_cut_pure,desc_cut,desc_cut_pure)

def to_count(item):
    id,name_cut,name_cut_pure,desc_cut,desc_cut_pure = item
    for i in name_cut_pure:
        yield ((id, i),1)
    for i in desc_cut_pure:
        yield ((id,i),1)

In [31]:
cur = mydb.cursor()

In [32]:
cur.reset()
cur.execute("SELECT id,name,description from items order by RAND() ")
result = cur.fetchall()
# print(result)

In [33]:
all_items_rdd = sc.parallelize(result)
# print(all_items_rdd.collect())

In [34]:

items_cut_rdd = all_items_rdd.map(cut_name_and_desc)
# print(rdd2.collect())

In [35]:
def f1(item):
    aa,c = item
    a,b = aa
    return a,b,c
word_count = items_cut_rdd.flatMap(to_count).reduceByKey(lambda a,b:a+b).map(f1)
# print(word_count.collect())

In [36]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("item_id", IntegerType(), True),
    StructField("key_word", StringType(), True),
    StructField("word_count", IntegerType(), True)
])
df1 = spark.createDataFrame(word_count, schema)
df1.createOrReplaceTempView("word_count")
df1.show()

+-------+--------+----------+
|item_id|key_word|word_count|
+-------+--------+----------+
|    406|      牛|         1|
|    406|  黑白花|         1|
|    406|    毛色|         1|
|    406|      强|         1|
|    218|      牛|         2|
|    218|      药|         1|
|    218|    用于|         1|
|     86|    细腻|         1|
|     86|    选用|         1|
|     86|    采用|         1|
|     86|    四溢|         1|
|     86|    调料|         1|
|    255|  牛初乳|         1|
|    255|    乳品|         1|
|    264|    安徽|         1|
|    131|      牛|         2|
|    131|    草原|         2|
|    131|    整体|         1|
|    131|  内蒙古|         1|
|    131|    必需|         1|
+-------+--------+----------+
only showing top 20 rows



In [46]:
result = spark.sql('''
    with t1 as(
        select 
            item_id,
            sum(word_count) as word_count
        from
            word_count
        group by 
            item_id
    )
    select
        word_count.item_id,
        key_word,
        sum(word_count.word_count) as count_word,
        t1.word_count as total_word
    from
        word_count
    left join
        t1 on word_count.item_id = t1.item_id
    group by 
        word_count.item_id ,key_word, t1.word_count
    order by
        count_word desc
''')
result.show()

+-------+--------+----------+----------+
|item_id|key_word|count_word|total_word|
+-------+--------+----------+----------+
|     38|  牛肉干|         2|         5|
|    307|    服务|         2|        12|
|     46|      黑|         2|        24|
|     14|    夏日|         2|        19|
|     46|    牛排|         2|        24|
|    225|  抗生素|         2|         7|
|    307|      牛|         2|        12|
|    233|      牛|         2|         5|
|     14|      黑|         2|        19|
|    248|    药品|         2|        13|
|     14|    牧场|         2|        19|
|    280|    治疗|         2|         8|
|     46|    吉林|         2|        24|
|    280|  胃肠炎|         2|         8|
|    161|  荷斯坦|         2|        11|
|    282|    服务|         2|        11|
|    186|  荷斯坦|         2|        11|
|    282|    检疫|         2|        11|
|     14|    母牛|         2|        19|
|    282|      牛|         2|        11|
+-------+--------+----------+----------+
only showing top 20 rows



In [38]:
# spark.sql("CREATE TABLE IF NOT EXISTS item_word_count (item_id INT, key_word STRING, word_count INT) USING hive")
# result = spark.sql('''
#     insert into table item_word_count select * from word_count
# ''')
# result.show()
df1.write.saveAsTable("item_word_count_test", mode='overwrite')

+-------+--------+----------+
|item_id|key_word|word_count|
+-------+--------+----------+
|    446|  黑白花|         1|
|     47|    牛油|         1|
|     47|      黑|         1|
|     47|      焖|         1|
|     16|    优质|         1|
|     16|     500|         1|
|     16|      黑|         1|
|     44|  牛肉干|         1|
|     44|    食客|         1|
|    400|    红牛|         1|
|    445|    岗牛|         1|
|    445|  黑白花|         1|
|    445|    种苗|         1|
|    445|    一号|         1|
|    445|    供应|         1|
|    334|    安置|         1|
|     57|    特产|         1|
|    390|    指导|         1|
|    390|    喂养|         1|
|    342|  牛肉干|         1|
|    182|      牛|         1|
|    182|  荷斯坦|         1|
|    113|    东北|         1|
|    114|      牛|         1|
|    105|    牛蹄|         1|
|    164|    黑牛|         1|
|    164|    东北|         1|
|    283|      牛|         1|
|    283|    配种|         1|
|     94|    香脆|         1|
+-------+--------+----------+
only showing top 30 rows

